In [23]:
from os import sys, path
sys.path.append(path.dirname(path.dirname(path.abspath('__file__'))))

In [24]:
import pprint
import h5py

In [25]:
import progressbar

In [26]:
from scoring.utils import *

In [27]:
df = pd.read_csv('/home/kasper/Dropbox/Scrapping/soccerway/csv/final_data_soccerway.csv', index_col='Unnamed: 0')
pred_data = pd.read_csv('../data/predict.csv', index_col='Unnamed: 0')
df_teams = pred_data[['home_team','away_team']].values

/home/kasper/PycharmProjects/venv/lib/python3.5/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [28]:
result_key = 'result_final'
n_states = 3

In [29]:
print(df_teams.shape)

(108, 2)


In [30]:
bar = progressbar.ProgressBar(widgets=[
    ' [', progressbar.Timer(), '] ',
    progressbar.Bar(),
    ' (', progressbar.ETA(), ') ',
])

ps = []
try:
    n_start=int(sys.argv[1])
    n_end=int(sys.argv[2])
except Exception as e:
#     print(e)
    n_start=10
    n_end=df_teams.shape[0]
print(n_start,n_end)
last_scores =  []
ps = []
for p, pair in bar(enumerate(df_teams[n_start:n_end, :],start=n_start)):
    champ, teams, teamh, teama, datah, dataa = get_team_data(p, pair)
    ht, at, htd, atd, last_score_home, last_score_away = get_ratings(datah, dataa, teamh, teama)
    last_scores.append([last_score_home, last_score_away])
    ps.append(p)

 [Elapsed Time: 0:00:00] |#                                  | (ETA:      N/A) 

10 108


 [Elapsed Time: 0:00:02] |#                                  | (Time: 0:00:02) 


In [32]:
ls = np.array(last_scores)
last_scores_df = pd.DataFrame(ls,columns=['EH','EA'], index=ps)
last_scores_df.head()
last_scores_df.to_csv('./elo_last.csv', index=False)

In [ ]:
# ###### generate dataset for classifier

# for p, pair in enumerate(df_teams[:2]):
#     print(pair)
# #     p = 0

#     h5f = h5py.File('./elo_temp/elo_pairs_'+str(p)+'.h5','r')

#     champ = list(h5f.keys())[0]

#     teams = list(h5f[champ].keys())

#     ixs = np.array([])
#     for team in teams[:]:    
#         ix = h5f[champ][team][:].T[:,0]
#         ixs = np.append(ixs, ix)
#     uni = np.unique(ixs).astype(int)

    

#     da = df.loc[uni][['home_team','away_team','result_final','over_under_0.5','over_under_1.5','over_under_2.5','over_under_3.5','over_under_4.5']]


#     elos =np.zeros((da.shape[0],2))
#     for i, (ix, row) in enumerate(da.iloc[:].iterrows()):
#         if i%1000==0:
#             print(i,da.shape[0])
#         try:
#             hdhome = h5f[champ][row.home_team][:].T
#             hdaway = h5f[champ][row.away_team][:].T
#             elohome = hdhome[np.where(hdhome==ix)[1],1]
#             eloaway = hdaway[np.where(hdaway==ix)[1],1]
#             elos[i,0] = elohome
#             elos[i,1] = eloaway
#         except Exception as e:
#             print(e)

#     elos_df = pd.DataFrame(elos, columns=['EH','EA'], index=da.index)

#     final = pd.concat([da,elos_df],axis=1)
# #     final = final.dropna()

#     final.to_csv('./elo_goals_data_for_classifier/data_'+str(p)+'.csv')
#     h5f.close()

# final.head()

# final = pd.read_csv('./data_'+str(p)+'.csv', index_col='Unnamed: 0')

# data = final.iloc[:]
# data.head(5)

# ht = final.home_team.values
# at = final.away_team.values
# all_teams = np.append(ht,at)
# un = np.unique(all_teams)

# le = preprocessing.LabelEncoder()
# le.fit(un)

# # transformed_observed_data_elo_ratings = le.transform(ys)

# final.columns

# final.loc[:,'home_team'] = le.transform(final.home_team.values)
# final.loc[:,'away_team'] =  le.transform(final.away_team.values)









# X = final.iloc[:10000][['home_team','away_team','EH','EA']].values
# y = final.iloc[:10000]['over_under_2.5'].values
# X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.75, test_size=0.25, random_state=42)

# tpot = TPOTClassifier(generations=5, population_size=13, verbosity=2)

# tpot.fit(X_train, y_train)

# print(tpot.score(X_test, y_test))

















# tpot.export('tpot_mnist_pipeline.py')

# # %load tpot_mnist_pipeline.py
# import numpy as np
# import pandas as pd
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.feature_selection import VarianceThreshold
# from sklearn.kernel_approximation import RBFSampler
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import make_pipeline, make_union
# from tpot.builtins import StackingEstimator
# from xgboost import XGBClassifier

# # NOTE: Make sure that the class is labeled 'target' in the data file
# tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
# features = tpot_data.drop('target', axis=1).values
# training_features, testing_features, training_target, testing_target = \
#             train_test_split(features, tpot_data['target'].values, random_state=42)

# # Score on the training set was:0.623731933095674
# exported_pipeline = make_pipeline(
#     VarianceThreshold(threshold=0.9000000000000001),
#     StackingEstimator(estimator=GradientBoostingClassifier(learning_rate=0.1, max_depth=8, max_features=0.05, min_samples_leaf=4, min_samples_split=15, n_estimators=100, subsample=0.8500000000000001)),
#     RBFSampler(gamma=0.1),
#     XGBClassifier(learning_rate=0.01, max_depth=6, min_child_weight=13, n_estimators=100, nthread=1, subsample=0.05)
# )

# exported_pipeline.fit(training_features, training_target)
# results = exported_pipeline.predict(testing_features)


# X = final.iloc[:][['home_team','away_team','EH','EA']].values
# y = final.iloc[:]['over_under_2.5'].values
# X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.75, test_size=0.25, random_state=42)



# gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1)

# gbm.fit(X_train, y_train)
# predictions = gbm.predict(X_test)



# pprint.pprint(metrics.classification_report(predictions, y_test))

# ac = metrics.accuracy_score(predictions, y_test)

# print(ac)

# metrics.confusion_matrix(predictions, y_test)


# conf_arr = metrics.confusion_matrix(predictions, y_test)

# norm_conf = []
# for i in conf_arr:
#     a = 0
#     tmp_arr = []
#     a = sum(i, 0)
#     for j in i:
#         tmp_arr.append(float(j)/float(a))
#     norm_conf.append(tmp_arr)

# fig = plt.figure()
# plt.clf()
# ax = fig.add_subplot(111)
# ax.set_aspect(1)
# res = ax.imshow(np.array(norm_conf), cmap=plt.cm.jet, 
#                 interpolation='nearest')

# width, height = conf_arr.shape

# for x in range(width):
#     for y in range(height):
#         ax.annotate(str(conf_arr[x][y]), xy=(y, x), 
#                     horizontalalignment='center',
#                     verticalalignment='center')

# cb = fig.colorbar(res)
# alphabet1 = ['Predicted-O','Predicted-U']
# alphabet2 = ['Actual-O','Actual-U']
# plt.xticks(range(width), alphabet2[:width])
# plt.yticks(range(height), alphabet1[:height])
# plt.savefig('confusion_matrix.png', format='png')





# from tpot import TPOTClassifier
# from sklearn.datasets import load_digits
# from sklearn.model_selection import train_test_split

# digits = load_digits()
# X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target,
#                                                     train_size=0.75, test_size=0.25)

# digits.data

# digits.target

# tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2)
# tpot.fit(X_train, y_train)



# elohome

# ar_data[0].astype(int)

# dfs[1]

# ar_data = h5f[champ][team][:].T[r]

# ar_data

# tot = pd.concat(dfs, axis=0, ignore_index=False)

# tot

# h5f.close()

# len(outcomes)

# ypred
# metrics.recall_score(y_pred=ypred, y_true=ytrue, pos_label=0)

# ypred

# ytrue
# transformed_observed_data_elo_ratings.shape

# bob_says = transformed_observed_data_elo_ratings.reshape(-1,1)[-470:]
# model = model.fit(bob_says)
# transformed_observed_data_elo_ratings.reshape(-1, 1)[-10:]

# bob_says

# h5f.close()

# print("Bob said:", ", ".join(map(lambda x: observations[int(x)], bob_says)))
# print("Alice Believes:", ", ".join(map(lambda x: states[x], alice_hears)))
# h5f.close()
# pxx = transition_matrix_pxx(data=df.loc[htd.index]['result_final'].dropna().to_frame(),result='result_final',n_states=3)

# pyx = emission_probabilities_pyx(x_df=df.loc[htd.index]['result_final'].dropna().to_frame(),x_label='result_final',hidden_states=3,y_df=htd)

# px = df.loc[htd.index]['result_final'].value_counts(normalize=True).sort_index()

# px

# pxx


# df.loc[htd.index]['result_final'].dropna().to_frame()

# a = df[((df.home_team==teamh) & (df.away_team==teama)) | ((df.home_team==teama)&(df.away_team==teamh))]

# a

# l = pd.concat([ht,at],axis=1)

# l.plot()
# # l.loc[a.index].plot()

# pd.concat([htd,atd],axis=1).plot()

# pd.concat([ht,at],axis=1).plot()

# ht.plot()
# at.plot()

# # df.loc[995088]

# # df[((df.home_team==teams[i]) | (df.away_team==teams[i])) & (df.championship==champ)]

# # df.loc[2054071]

# import itertools
# import math
# import trueskill
# def win_probability(team1, team2):
#     delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
#     sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
#     size = len(team1) + len(team2)
#     denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
#     ts = trueskill.global_env()
#     return ts.cdf(delta_mu / denom)

# BETA = ts.BETA
# def win_probability(a, b):                                                      
#     deltaMu = sum([x.mu for x in a]) - sum([x.mu for x in b])                   
#     sumSigma = sum([x.sigma ** 2 for x in a]) + sum([x.sigma ** 2 for x in b])  
#     playerCount = len(a) + len(b)                                               
#     denominator = math.sqrt(playerCount * (BETA * BETA) + sumSigma)             
#     return cdf(deltaMu / denominator)  

# teams = np.arange(100)
# ratings = [Rating() for i in range(100)]
# for i in range(100):
#     choice = np.random.choice(teams,size=2,replace=True)
#     a = ratings[choice[0]]
#     b = ratings[choice[1]]
#     q = quality_1vs1(a, b)
#     wp = win_probability([a],[b])
#     print(wp)
#     a, b = ts.rate_1vs1(a, b, drawn=False)
    
#     ratings[choice[0]] = a
#     ratings[choice[1]] = b

# mus = [r.mu for r in ratings]
# sigma = [r.sigma for r in ratings]

# ### sources

# #https://stackoverflow.com/questions/5821125/how-to-plot-confusion-matrix-with-string-axis-rather-than-integer-in-python

# # https://stackoverflow.com/questions/5375624/a-decorator-that-profiles-a-method-call-and-logs-the-profiling-result

# def profileit(prof_fname, sort_field='cumtime'):
#     """
#     Parameters
#     ----------
#     prof_fname
#         profile output file name
#     sort_field
#         "calls"     : (((1,-1),              ), "call count"),
#         "ncalls"    : (((1,-1),              ), "call count"),
#         "cumtime"   : (((3,-1),              ), "cumulative time"),
#         "cumulative": (((3,-1),              ), "cumulative time"),
#         "file"      : (((4, 1),              ), "file name"),
#         "filename"  : (((4, 1),              ), "file name"),
#         "line"      : (((5, 1),              ), "line number"),
#         "module"    : (((4, 1),              ), "file name"),
#         "name"      : (((6, 1),              ), "function name"),
#         "nfl"       : (((6, 1),(4, 1),(5, 1),), "name/file/line"),
#         "pcalls"    : (((0,-1),              ), "primitive call count"),
#         "stdname"   : (((7, 1),              ), "standard name"),
#         "time"      : (((2,-1),              ), "internal time"),
#         "tottime"   : (((2,-1),              ), "internal time"),
#     Returns
#     -------
#     None

#     """
#     def actual_profileit(func):
#         def wrapper(*args, **kwargs):
#             prof = cProfile.Profile()
#             retval = prof.runcall(func, *args, **kwargs)
#             stat_fname = '{}.stat'.format(prof_fname)
#             prof.dump_stats(prof_fname)
#             print_profiler(prof_fname, stat_fname, sort_field)
#             print('dump stat in {}'.format(stat_fname))
#             return retval
#         return wrapper
#     return actual_profileit


# def print_profiler(profile_input_fname, profile_output_fname, sort_field='cumtime'):
#     import pstats
#     with open(profile_output_fname, 'w') as f:
#         stats = pstats.Stats(profile_input_fname, stream=f)
#         stats.sort_stats(sort_field)
#         stats.print_stats()

#         import cProfile
# import functools

# def profileit(func):
#     @functools.wraps(func)  # <-- Changes here.
#     def wrapper(*args, **kwargs):
#         datafn = func.__name__ + ".profile" # Name the data file sensibly
#         prof = cProfile.Profile()
#         retval = prof.runcall(func, *args, **kwargs)
#         prof.dump_stats(datafn)
#         print(prof.__str__)
#         return retval

#     return wrapper

# @profilei



# test()

# en1 = cluster[0]

# async_results = []

# a = view.apply_async(get_ratings,datah, dataa, teamh, teama)


# async_results.append(a)
# # cluster.wait(async_results)
# cluster.wait_interactive(async_results) 

# async_results[0].error

# # %%timeit

# for p, pair in enumerate(df_teams[:10]):
#     something_1(p,pair)





# # %time
# async_results = []
# for p, pair in enumerate(df_teams[:10]):
#     print(pair)
#     ar = view.apply_async(something_1, p, pair)
#     async_results.append(ar)
# # cluster.wait(async_results)
# cluster.wait_interactive(async_results) 

# r = async_results[0]

# r.done()

# r.error